# Text Summarization

Text summarization refers to the technique of shortening long pieces of text. The intention is to create a coherent and fluent summary having only the main points outlined in the document.
Automatic text summarization is a common problem in machine learning and natural language processing (NLP).

There are broadly two different approaches that are used for text summarization:

* Extractive Summarization
* Abstractive Summarization


In this notebook, we will build an extraction based text summarizer using python

### Extractive Summarization
The name gives away what this approach does. We identify the important sentences or phrases from the original text and extract only those from the text. Those extracted sentences would be our summary. The below diagram illustrates extractive summarization:

![Extractive Summarization](images/Extractive_Summarization.png)

# TextRank Algorithm

TextRank is an automatic summarisation technique. Graph-based ranking algorithms are a way for deciding the importance of a vertex within a graph, based on global information recursively drawn from the entire graph. The basic idea implemented by a graph-based ranking model is that of voting or recommendation.

![TextRank](images/text_rank.png)

# Text Summarization using pytextrank
https://github.com/DerwenAI/pytextrank

In [0]:
import spacy
import pytextrank

# example text
text = "Compatibility of systems of linear constraints over the set of natural numbers. Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered. Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given. These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types."

# load a spaCy model, depending on language, scale, etc.
nlp = spacy.load("en_core_web_sm")

# add PyTextRank to the spaCy pipeline
tr = pytextrank.TextRank()
nlp.add_pipe(tr.PipelineComponent, name="textrank", last=True)

doc = nlp(text)

# examine the top-ranked phrases in the document
for p in doc._.phrases:
    print("{:.4f} {:5d}  {}".format(p.rank, p.count, p.text))
    print(p.chunks)

0.1567     1  minimal generating sets
[minimal generating sets]
0.1371     4  systems
[systems, systems, systems, a system]
0.1178     3  solutions
[solutions, solutions, solutions]
0.1164     1  linear diophantine equations
[linear Diophantine equations]
0.1077     1  nonstrict inequations
[nonstrict inequations]
0.1050     1  mixed types
[mixed types]
0.1044     1  strict inequations
[strict inequations]
0.1000     1  a minimal supporting set
[a minimal supporting set]
0.0979     1  linear constraints
[linear constraints]
0.0919     1  upper bounds
[Upper bounds]
0.0913     1  a minimal set
[a minimal set]
0.0804     1  components
[components]
0.0797     1  natural numbers
[natural numbers]
0.0797     1  algorithms
[algorithms]
0.0782     1  all the considered types systems
[all the considered types systems]
0.0768     1  diophantine
[Diophantine]
0.0697     2  compatibility
[Compatibility, compatibility]
0.0693     1  construction
[construction]
0.0668     1  the set
[the set]
0.062

Construct a list of the sentence boundaries with a phrase vector (initialized to empty set) for each...

In [0]:
sent_bounds = [ [s.start, s.end, set([])] for s in doc.sents ]
sent_bounds

[[0, 13, set()], [13, 33, set()], [33, 61, set()], [61, 91, set()]]

Iterate through the top-ranked phrases, added them to the phrase vector for each sentence...

In [0]:
limit_phrases = 4

phrase_id = 0
unit_vector = []

for p in doc._.phrases:
    print(phrase_id, p.text, p.rank)
    
    unit_vector.append(p.rank)
    
    for chunk in p.chunks:
        print(" ", chunk.start, chunk.end)
        
        for sent_start, sent_end, sent_vector in sent_bounds:
            if chunk.start >= sent_start and chunk.start <= sent_end:
                print(" ", sent_start, chunk.start, chunk.end, sent_end)
                sent_vector.add(phrase_id)
                break

    phrase_id += 1

    if phrase_id == limit_phrases:
        break

0 minimal generating sets 0.15674194595995644
  48 51
  33 48 51 61
1 systems 0.13714815025016455
  2 3
  0 2 3 13
  57 58
  33 57 58 61
  86 87
  61 86 87 91
  17 19
  13 17 19 33
2 solutions 0.11782962717984638
  42 43
  33 42 43 61
  52 53
  33 52 53 61
  74 75
  61 74 75 91
3 linear diophantine equations 0.11639333486895956
  20 23
  13 20 23 33


Let's take a look at the results...

In [0]:
sent_bounds

[[0, 13, {1}], [13, 33, {1, 3}], [33, 61, {0, 1, 2}], [61, 91, {1, 2}]]

In [0]:
for sent in doc.sents:
    print(sent)

Compatibility of systems of linear constraints over the set of natural numbers.
Criteria of compatibility of a system of linear Diophantine equations, strict inequations, and nonstrict inequations are considered.
Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given.
These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types.


In [0]:
unit_vector

[0.15674194595995644,
 0.13714815025016455,
 0.11782962717984638,
 0.11639333486895956]

In [0]:
sum_ranks = sum(unit_vector)
unit_vector = [ rank/sum_ranks for rank in unit_vector ]

unit_vector

[0.2967961945055862,
 0.25969467731457346,
 0.223114398209173,
 0.22039472997066747]

Iterate through each sentence, calculating its euclidean distance from the unit vector...

In [0]:
from math import sqrt

sent_rank = {}
sent_id = 0

for sent_start, sent_end, sent_vector in sent_bounds:
    print(sent_vector)
    sum_sq = 0.0
    
    for phrase_id in range(len(unit_vector)):
        print(phrase_id, unit_vector[phrase_id])
        
        if phrase_id not in sent_vector:
            sum_sq += unit_vector[phrase_id]**2.0

    sent_rank[sent_id] = sqrt(sum_sq)
    sent_id += 1

print(sent_rank)

{1}
0 0.2967961945055862
1 0.25969467731457346
2 0.223114398209173
3 0.22039472997066747
{1, 3}
0 0.2967961945055862
1 0.25969467731457346
2 0.223114398209173
3 0.22039472997066747
{0, 1, 2}
0 0.2967961945055862
1 0.25969467731457346
2 0.223114398209173
3 0.22039472997066747
{1, 2}
0 0.2967961945055862
1 0.25969467731457346
2 0.223114398209173
3 0.22039472997066747
{0: 0.43178912996980673, 1: 0.37130582511083665, 2: 0.22039472997066747, 3: 0.36967799240939564}


Sort the sentence indexes in descending order

In [0]:
from operator import itemgetter

sorted(sent_rank.items(), key=itemgetter(1))

[(2, 0.22039472997066747),
 (3, 0.36967799240939564),
 (1, 0.37130582511083665),
 (0, 0.43178912996980673)]

Extract the sentences with the lowest distance, up to the limite requested...

In [0]:
limit_sentences = 2

sent_text = {}
sent_id = 0

for sent in doc.sents:
    sent_text[sent_id] = sent.text
    sent_id += 1

num_sent = 0

for sent_id, rank in sorted(sent_rank.items(), key=itemgetter(1)):
    print(sent_id, sent_text[sent_id])
    num_sent += 1
    
    if num_sent == limit_sentences:
        break

2 Upper bounds for components of a minimal set of solutions and algorithms of construction of minimal generating sets of solutions for all types of systems are given.
3 These criteria and the corresponding algorithms for constructing a minimal supporting set of solutions can be used in solving all the considered types systems and systems of mixed types.
